In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.5188, 0.8188, 0.9268],
        [0.7047, 0.8714, 0.2704],
        [0.8120, 0.7637, 0.7288],
        [0.2592, 0.5980, 0.8463],
        [0.3507, 0.1861, 0.6446]])


In [3]:
import torch
torch.cuda.is_available()

False

In [4]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [5]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [6]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [10]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.309198  [    0/60000]
loss: 2.296366  [ 6400/60000]
loss: 2.282508  [12800/60000]
loss: 2.276676  [19200/60000]
loss: 2.255650  [25600/60000]
loss: 2.242434  [32000/60000]
loss: 2.246337  [38400/60000]
loss: 2.221770  [44800/60000]
loss: 2.202351  [51200/60000]
loss: 2.178186  [57600/60000]
Test Error: 
 Accuracy: 43.3%, Avg loss: 2.175334 

Epoch 2
-------------------------------
loss: 2.181892  [    0/60000]
loss: 2.171413  [ 6400/60000]
loss: 2.122653  [12800/60000]
loss: 2.139632  [19200/60000]
loss: 2.090736  [25600/60000]
loss: 2.040882  [32000/60000]
loss: 2.070450  [38400/60000]
loss: 1.997266  [44800/60000]
loss: 1.988928  [51200/60000]
loss: 1.935115  [57600/60000]
Test Error: 
 Accuracy: 53.7%, Avg loss: 1.927672 

Epoch 3
-------------------------------
loss: 1.951774  [    0/60000]
loss: 1.927298  [ 6400/60000]
loss: 1.816237  [12800/60000]
loss: 1.864127  [19200/60000]
loss: 1.759596  [25600/60000]
loss: 1.699466  [32000/600

In [11]:
torch.save(model, './model.pth')

In [12]:
model = torch.load('model.pth').to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [13]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.188636  [    0/60000]
loss: 1.185582  [ 6400/60000]
loss: 1.009951  [12800/60000]
loss: 1.148431  [19200/60000]
loss: 1.026239  [25600/60000]
loss: 1.044956  [32000/60000]
loss: 1.085175  [38400/60000]
loss: 1.020396  [44800/60000]
loss: 1.054685  [51200/60000]
loss: 0.986616  [57600/60000]
Test Error: 
 Accuracy: 65.4%, Avg loss: 1.004159 

Epoch 2
-------------------------------
loss: 1.063237  [    0/60000]
loss: 1.078318  [ 6400/60000]
loss: 0.887260  [12800/60000]
loss: 1.048140  [19200/60000]
loss: 0.929127  [25600/60000]
loss: 0.945044  [32000/60000]
loss: 1.000181  [38400/60000]
loss: 0.939779  [44800/60000]
loss: 0.969484  [51200/60000]
loss: 0.913254  [57600/60000]
Test Error: 
 Accuracy: 66.8%, Avg loss: 0.925638 

Epoch 3
-------------------------------
loss: 0.971832  [    0/60000]
loss: 1.004339  [ 6400/60000]
loss: 0.799624  [12800/60000]
loss: 0.978001  [19200/60000]
loss: 0.864476  [25600/60000]
loss: 0.871690  [32000/600